In [6]:
"""Bionic bot"""

import operator
import os
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.load import dumps
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

In [17]:
# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
llm = ChatOpenAI(temperature=0, streaming=True, model="gpt-4-turbo")


class ChatRouter(BaseModel):
    """Call this if you are able to route the user to the appropriate representative."""

    choice: str = Field(description="should be one of: user, workflow")


system_message = """
You are a `digital double` of the user.  You are an extension and reflection of the user.

You should interact tersely with them to try to figure out how you can route their message:

- User understanding: learn about the user, inform them of what you have learned about them, adapt to their styles and preferences. Call the router with `user`
- Workflows: your main job is to offload tasks from the user.  You do this with workflows.  If the user is asking about creating, updating, performing, or the status of a task or workflow, then call the router with `workflow`

Ask youself: is the user message about the user?  something the like, something they've done or want to do (aspirations), dreams, experiences, preferences, etc.?  If so, then route to user understanding ('user').
Otherwise, if the user message is not about themselves (user understanding) or about tasks (workflows), then respond directing them to one of these topics.
"""


def get_messages(messages):
    return [SystemMessage(content=system_message)] + messages

In [18]:
chain = get_messages | llm.bind_tools([ChatRouter])

In [19]:
msgs = [HumanMessage(content="i rarely eat vegetables")]
chain.invoke(msgs)

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Qmpp9nUGxPqJMStNBfX5fxYR', 'function': {'arguments': '{"choice":"user"}', 'name': 'ChatRouter'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-0bbb40cf-7a1d-40db-b4f2-6b582e436cc4-0', tool_calls=[{'name': 'ChatRouter', 'args': {'choice': 'user'}, 'id': 'call_Qmpp9nUGxPqJMStNBfX5fxYR'}])

In [20]:
msgs = [HumanMessage(content="what's the status of the TPS report?")]
chain.invoke(msgs)

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ZBsBn5lVPWPToklgpqaDMYPJ', 'function': {'arguments': '{"choice":"workflow"}', 'name': 'ChatRouter'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-b5a26d47-3197-4754-ad42-ce81ff1fc6c3-0', tool_calls=[{'name': 'ChatRouter', 'args': {'choice': 'workflow'}, 'id': 'call_ZBsBn5lVPWPToklgpqaDMYPJ'}])

In [21]:
msgs = [HumanMessage(content="I am a professional software developer")]
chain.invoke(msgs)

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_p0DfBF9H2o5usmqxpUSwZNa2', 'function': {'arguments': '{"choice":"user"}', 'name': 'ChatRouter'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-2263dde9-9afb-477e-abc6-b32d14aa6827-0', tool_calls=[{'name': 'ChatRouter', 'args': {'choice': 'user'}, 'id': 'call_p0DfBF9H2o5usmqxpUSwZNa2'}])

In [22]:
msgs = [HumanMessage(content="What is the weather going to be like tomorrow?")]
chain.invoke(msgs)

AIMessage(content="I'm here to help with tasks related to workflows or to learn more about you. For weather updates, you might want to check a weather website or app. Is there anything else I can assist you with related to your tasks or personal preferences?", response_metadata={'finish_reason': 'stop'}, id='run-325a871d-fc03-490d-9d24-353eb70b5528-0')